# Part I. ETL Pipeline for Pre-Processing the Files
### we put all the lines from all the files in folder event_data in one single file

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

/home/workspace


In [3]:
# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'
filepath

'/home/workspace/event_data'

In [4]:
# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

In [5]:
file_path_list[0:5]

['/home/workspace/event_data/2018-11-27-events.csv',
 '/home/workspace/event_data/2018-11-04-events.csv',
 '/home/workspace/event_data/2018-11-07-events.csv',
 '/home/workspace/event_data/2018-11-09-events.csv',
 '/home/workspace/event_data/2018-11-19-events.csv']

In [6]:
#!cat /home/workspace/event_data/2018-11-27-events.csv

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [7]:
# initiating an empty list of rows that will be generated from each file
# each element of the lis is a row from the files in "file_path_list"

full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [8]:
full_data_rows_list[:2]

[['Barry Tuckwell/Academy of St Martin-in-the-Fields/Sir Neville Marriner',
  'Logged In',
  'Mohammad',
  'M',
  '0',
  'Rodriguez',
  '277.15873',
  'paid',
  'Sacramento--Roseville--Arden-Arcade, CA',
  'PUT',
  'NextSong',
  '1.54051E+12',
  '961',
  'Horn Concerto No. 4 in E flat K495: II. Romance (Andante cantabile)',
  '200',
  '1.54328E+12',
  '88'],
 ['Jimi Hendrix',
  'Logged In',
  'Mohammad',
  'M',
  '1',
  'Rodriguez',
  '239.82975',
  'paid',
  'Sacramento--Roseville--Arden-Arcade, CA',
  'PUT',
  'NextSong',
  '1.54051E+12',
  '961',
  'Woodstock Inprovisation',
  '200',
  '1.54328E+12',
  '88']]

In [9]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


In [10]:
!cat event_datafile_new.csv

In [11]:
df = pd.read_csv('event_datafile_new.csv', nrows=5)
df.head()


,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Mohammad,M,0,Rodriguez,277.15873,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Horn Concerto No. 4 in E flat K495: II. Romanc...,88
1,Jimi Hendrix,Mohammad,M,1,Rodriguez,239.82975,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Woodstock Inprovisation,88
2,Building 429,Mohammad,M,2,Rodriguez,300.61669,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Majesty (LP Version),88
3,The B-52's,Gianna,F,0,Jones,321.54077,free,"New York-Newark-Jersey City, NY-NJ-PA",107,Love Shack,38
4,Die Mooskirchner,Gianna,F,1,Jones,169.29914,free,"New York-Newark-Jersey City, NY-NJ-PA",107,Frisch und g'sund,38


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 11 columns):
artist           5 non-null object
firstName        5 non-null object
gender           5 non-null object
itemInSession    5 non-null int64
lastName         5 non-null object
length           5 non-null float64
level            5 non-null object
location         5 non-null object
sessionId        5 non-null int64
song             5 non-null object
userId           5 non-null int64
dtypes: float64(1), int64(3), object(7)
memory usage: 520.0+ bytes


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now we are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [13]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster

try: 
    cluster = Cluster(['127.0.0.1']) #If you have a locally installed Apache Cassandra instance
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [14]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [15]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

## Now we create tables to run the following queries. Remember, with Apache Cassandra we model the database tables on the queries we want to run.

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

#### For the table songs_by_user_session, the columns sessionId and iteminSession were used as Composite Partition Key because the queries will filter by the columns sessionId and iteminSession. 

In [16]:
query = "CREATE TABLE IF NOT EXISTS songs_by_user_session "
query = query + "(sessionId INT , \
                iteminSession INT, \
                artist TEXT, \
                song TEXT, \
                length FLOAT, \
                PRIMARY KEY ((sessionId,iteminSession)))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [17]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query= "INSERT INTO songs_by_user_session (sessionId , iteminSession, artist, song, length)"
        query= query+ " VALUES (%s, %s, %s, %s, %s)"
        try:
            session.execute(query, (int(line[3]),int(line[8]),line[0],line[9],float(line[5])))
        except Exception as e:
            print(e)

#### The following query will verify that the data have been inserted correctly into the table

In [18]:
query = "SELECT artist,song,length FROM songs_by_user_session WHERE sessionId = 338 AND iteminSession = 4 LIMIT 5"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row)

### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

#### For the table artist_song_user_by_userid_sessionid, the columns userid and sessionid were used as Composite Partition Key because the queries will filter by the columns userId and sessionId. The column  iteminSession was used as clustering columns to help make up a unique key.

In [19]:
query = "CREATE TABLE IF NOT EXISTS artist_song_user_by_userid_sessionId "
query = query + "(userId INT, \
                sessionId INT, \
                itemInSession INT, \
                artist TEXT, \
                song TEXT, \
                firstName TEXT, \
                lastName TEXT, \
                PRIMARY KEY ((userid,sessionid),iteminSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query= "INSERT INTO artist_song_user_by_userid_sessionId (userId,sessionId,itemInSession,artist,song,firstName,lastName)"
        query= query+ " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        try:
            session.execute(query, (int(line[10]),int(line[8]),int(line[3]),line[0],line[9],line[1],line[4]))
        except Exception as e:
            print(e)
            break


query = "SELECT artist,song,firstName,lastName FROM artist_song_user_by_userid_sessionId WHERE userid = 10 AND sessionid = 182 LIMIT 10"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)

Row(artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')



    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




#### For the table user_by_song, the column song was used as Partition Key because the queries will filter by the columns song. The column  userdId was used as clustering columns to help make up a unique key because the same song can have been played by more than just one user.

In [20]:
query = "CREATE TABLE IF NOT EXISTS user_by_song "
query = query + "(song TEXT, \
                userdId INT, \
                firstName TEXT, \
                lastName TEXT, \
                PRIMARY KEY (song,userdId))"
try:
    session.execute(query)
except Exception as e:
    print(e)

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query= "INSERT INTO user_by_song (song,userdId,firstName,lastName)"
        query= query+ " VALUES ( %s, %s, %s, %s)"
        try:
            session.execute(query, (line[9],int(line[10]),line[1],line[4]))
        except Exception as e:
            print(e)
            break

query = "SELECT userdId,firstName,lastName FROM user_by_song WHERE song='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)

Row(userdid=29, firstname='Jacqueline', lastname='Lynch')
Row(userdid=80, firstname='Tegan', lastname='Levine')
Row(userdid=95, firstname='Sara', lastname='Johnson')


### Drop the tables before closing out the sessions

In [21]:
query = 'drop table songs_by_user_session'
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = 'drop table artist_song_user_by_userid_sessionId'
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    

query = 'drop table user_by_song'
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [22]:
session.shutdown()
cluster.shutdown()